In [ ]:
from pywebio.input import input, FLOAT
from pywebio.output import put_text
from pywebio.input import *
from pywebio.output import *

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split 

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import plot_confusion_matrix

from heapq import nlargest


In [ ]:
def readFile(diag_fn, labs_fn):
    diag_file = open(diag_fn)
    labs_fn = open(labs_fn)
    
    diagnosis_admissions_table = pd.read_table(diag_fn)
    
    labs_table = pd.read_table(labs_fn)
    labs_table['PatientID'] =  labs_table['PatientID'] # + '_' + labs_table['AdmissionID'].astype(str)
            
    return diagnosis_admissions_table, labs_table

In [ ]:
diag_table, labs_table = readFile('AdmissionsDiagnosesCorePopulatedTable_100_new.txt', 'LabsCorePopulatedTable_100_new.txt')

In [4]:
labs_table = labs_table.drop('AdmissionID', axis=1)

In [5]:
grouped_labs = labs_table.groupby(['PatientID', 'LabName']).aggregate(np.mean)

In [6]:
labs_dict = {'pat 1':[1, 2, 3]}

In [7]:
for label, value in grouped_labs.iterrows():

    patient_id = label[0]
    
    if patient_id in labs_dict.keys():
        # print(patient_id + ' already exists')
        labs_dict[patient_id].append(value[0])
    else:
        
        labs_dict[patient_id] = []
        labs_dict[patient_id].append(value[0])


In [8]:
len(labs_dict.keys())

101

In [9]:
def show_instructions():
    
    popup('Welcome.', [
    put_html('Please download an Electronic Health Record Template for CBC, Metabolic, and Urinalysis Lab Results. Once you upload your patient\'s records, you will receive a diagnosis suggestion based on 5 patients with the most similar medical history.'),
        
    put_file(name='CBC Lab Results', content=b'Absolute Lymphocytes:\nAbsolute Neutrophil:\nBasophils:\nEosinophils:\nHematocrit:\nHemoglobin:\nLymphocytes:\nMCH:\nMCHC:\nMean Corpuscular Volume:\nMonocytes:\nNeutrophils:\nPlatelet Count:\nRDW:\nRed Blood Cell Count:\nWhite Blood Cell Count:', label='Download CBC Lab Results Template'),
    put_file(name='Metabolic Lab Results', content=b'Albumin:\nALK PHOS:\nALT/SGPT:\nANION GAP:\nAST/SGOT:\nBILI Total:\nBUN:\nCalcium\nCarbon Dioxide:\nChloride:\nCreatinine:\nGlucose:\nPotassium:\nSodium:\nTotal Protein:', label='Download Metabolic Lab Results Template'),
    put_file(name='Urinalysis Lab Results', content=b'pH:\nRed Blood Cells:\nSpecific Gravity:\nWhite Blood Cells:', label='Download Urinalysis Lab Results Template'),

    put_button('Got it!', onclick=close_popup)])

In [10]:
# VARIABLES

cbc_results_file = None
metab_results_file = None
uri_results_file = None

cbc_labs = []
metabolic_labs = []
urinalysis_lab = []

file_names = []

In [11]:
# ---- NOT USING RN ----

def show_upload_tests(btn_val):
    
    if btn_val == 'CBC':
        f = file_upload("Patient's CBC Lab Results:", accept="txt/*", multiple=False)
        cbc_results_file = f['filename']
        # print('... uploading cbc...')
        
    if btn_val == 'Metabolic':
        f = file_upload("Patient's Metabolic Lab Results:", accept="txt/*", multiple=False)['filename']
        metab_results_file = f['filename']
        # print('... uploading metab...')
        
    if btn_val == 'Urinalysis':
        f = file_upload("Patient's Urinalysis Lab Results:", accept="txt/*", multiple=False)['filename']
        uri_results_file = f['filename']
        # print('... uploading uri...')
        

In [12]:


def read_labs(cbc_labs_fn, metabolic_labs_fn, urinalysis_labs_fn): 
        
    with open(str(cbc_labs_fn)) as cbc_f:
        for line in cbc_f:
            # print(line.split(':')[1]) # lab value
            string_val = line.split(':')[1]
            cbc_labs.append(float(string_val))
            
    with open(metabolic_labs_fn) as metabolic_f:
        for line in metabolic_f:
            # print(line.split(':')[1]) # lab value
            string_val = line.split(':')[1]
            metabolic_labs.append(float(string_val))
            
    with open(urinalysis_labs_fn) as urinalysis_f:
        for line in urinalysis_f:
            # print(line.split(':')[1]) # lab value
            string_val = line.split(':')[1]
            urinalysis_lab.append(float(string_val))

In [13]:
def combine_all_labs(cbc_labs, metab_labs, uri_labs):
    
    all_labs = []
    
    for elem in cbc_labs:
        all_labs.append(elem)
        
    for elem in metab_labs:
        all_labs.append(elem)
        
    for elem in uri_labs:
        all_labs.append(elem)
        
    return all_labs

In [14]:
def compare_similar(new_patient_labs):

    sim_scores = {}

    for patient in labs_dict:
          
        db_pat_labs = np.array(labs_dict[patient])
        new_pat_labs = np.array(new_patient_labs)
        
        
        if(len(db_pat_labs) == len(new_pat_labs)):
            sim_score = np.mean(abs(db_pat_labs-new_pat_labs))
            sim_scores[patient] = sim_score
        
       
        # --- sim_score = np.mean(abs(db_pat_labs-new_pat_labs))
        # sim_score = 1 - np.mean(abs((np.array(labs_dict[patient]) - np.array(new_patient_labs))))
        # --- sim_scores[patient] = sim_score
        # print(len(labs_dict[patient]))
        # print(len(new_patient_labs))
        
        # print(sim_score)
        
    # return Nmaxelements(sim_scores, 5) # 5 most similar scores
    return sim_scores
        

In [15]:


def show_upload_options():
    files = file_upload("Upload labs:", accept="text/*", multiple=True)
    for file in files:
        file_names.append(file['filename'])
            
            
    for file in file_names:
        if 'cbc' in file:
            cbc_results_file = file
        if 'metab' in file:
            metab_results_file = file    
        if 'uri' in file:
            uri_results_file = file
    
    return cbc_results_file, metab_results_file, uri_results_file
    print(cbc_results_file + ', ' + metab_results_file + ', ' + uri_results_file)

In [16]:
if __name__ == '__main__':
    
    show_instructions()
    
    txt = put_text('Please click on a lab test to enter specific results. Once you submit these values, you will receive a diagnosis suggestion based on 5 patients with the most similar medical history.')    

    # put_buttons(['CBC', 'Metabolic', 'Urinalysis'], onclick=show_upload_tests)
    cbc, metab, uri = show_upload_options()
    
    read_labs(cbc, metab, uri)
    # read_labs(cbc_results_file, metab_results_file, uri_results_file)
    
    all_new_pat_labs = combine_all_labs(cbc_labs, metabolic_labs, urinalysis_lab)
    scores_dict = compare_similar(all_new_pat_labs)
    # print(scores_dict)
    
    most_sim_patients = nlargest(5, scores_dict, key = scores_dict.get)
    # print(most_sim_patients)
    
    
    for pat_id in most_sim_patients:
        
        print(pat_id + ':\n')
        indexes = list(diag_table[diag_table['PatientID'] == str(pat_id)].index)
        
        for index in indexes:
            
            print(diag_table.loc[index]['PrimaryDiagnosisDescription'])
            
        print('\n')

    

8856096E-E59C-4156-A767-C091AF799C80:

Osteitis deformans in neoplastic diseases


C2CCB1AB-6633-4CB3-B4E8-157E6FB02376:

Primary disorders of muscles
Antepartum hemorrhage with disseminated intravascular coagulation, third trimester


B2EB15FA-5431-4804-9309-4215BDC778C0:

Rheumatoid arthritis with rheumatoid factor of left elbow without organ or systems involvement
Mastoiditis in infectious and parasitic diseases classified elsewhere, bilateral
Chronic lymphocytic leukemia of B-cell type


0BC491C5-5A45-4067-BD11-A78BEA00D3BE:

Malignant neoplasm of pituitary gland
Schizoaffective disorder, bipolar type


B7E9FC4C-5182-4A34-954E-CEF5FC07E96D:

Type 1 diabetes mellitus with periodontal disease


